# Team Lambda Capstone Project

Benjamin Houldridge

Catherine Laguna

Elija Martin

##Problem Definition

##Data Collection and Sorces

### Imports

In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
import sklearn.model_selection as model_selection
from IPython.display import Image
import statsmodels.api as sm
from sklearn import preprocessing
import json
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pickle
from textblob import TextBlob as tb
from textblob import Word

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


###Read In Datasets

In [ ]:
lib1720_path = '/content/drive/My Drive/Team Lambda/Data/2017-2019.csv'
lib19_path = '/content/drive/My Drive/Team Lambda/Data/2019.csv'
lib20_path = '/content/drive/My Drive/Team Lambda/Data/2020.csv'
lib17_path = '/content/drive/My Drive/Team Lambda/Data/2017.csv'
lib1417_path = '/content/drive/My Drive/Team Lambda/Data/2014-2017.csv'

In [ ]:
nlp1720 = pd.read_csv(lib1720_path,encoding = "ISO-8859-1")

In [ ]:
nlp19 = pd.read_csv(lib19_path, encoding = "ISO-8859-1")

In [ ]:
nlp20 = pd.read_csv(lib20_path, encoding = "ISO-8859-1")

In [ ]:
nlp17 = pd.read_csv(lib17_path, encoding = "ISO-8859-1")

In [ ]:
nlp1417 = pd.read_csv(lib1417_path, encoding = "ISO-8859-1")

##Data Cleaning

nlp17 and npl1417 data all comes from the fullstack web development couse. This information was not included in the datasets and so needed to be added.

In [ ]:
nlp17['Which bootcamp did you attend?'] = 'Fullstack Web Development'
nlp1417['Which bootcamp did you attend?'] = 'Fullstack Web Development'

Some rows where missing information about which bootcamp was attended. Andrea Sisneros Wichman provided the missing information.

In [ ]:
nlp19.loc[30,'Which bootcamp did you attend?'] = 'Fullstack Web Development'
nlp20.loc[23,'Which bootcamp did you attend?'] = 'Internet of Things (IoT)'
nlp20.loc[25,'Which bootcamp did you attend?'] = 'Internet of Things (IoT)'
nlp20.loc[42,'Which bootcamp did you attend?'] = 'Digital Media'
nlp20.loc[54,'Which bootcamp did you attend?'] = 'Fullstack Web Development'

###Synchronization of Column Names

To create unified column names that will adhere to each question, the column names for each dataset will be redefined. Below is a set of dictionaries that map the old column names to new codes that represent each unique question. The first two smaller dictionaries exist because some of the data sets share the same column names but they correspond to different questions. The two smaller dictionaries will change a few of the names on their respective datasets then the large dictionary will be ran on all datasets to produce unified column names.

In [ ]:
column_names_mapped_nlp19 = {
    'Unnamed: 35': 'p14_s_r',
    'Unnamed: 43': 'qsue_r',
    'Unnamed: 47': 'qstem_r'
}

column_names_mapped_nlp20 = {
    'Unnamed: 35': 'p14_s_r',
    'Unnamed: 37': 'q14_1_r',
    'Unnamed: 43': 'qsue_r',
    'Unnamed: 47': 'qeste_r'
}

column_names_mapped_nlp1417 = {
    'Unnamed: 35': 'q_s_5_r',
    'Unnamed: 37': 'q_s_6_r'
}

column_names_mapped_nlp17 = {
    'Unnamed: 13': 'q4_r',
    'Unnamed: 15': 'q5_r',
    'Unnamed: 17': 'q6_r',
    'Unnamed: 19': 'q7_r',
    'Unnamed: 21': 'q8_r',
    'Unnamed: 29': 'q12_r',
    'Unnamed: 31': 'q1_r'
}

In [ ]:
column_names_mapped = {
    'Which bootcamp did you attend?':'q1',
    'Would you recommend this course to others?':'q1_1',
    'Unnamed: 10':'q1_r',
    'I understood the goals and objectives of this course when it began:':'q2',
    'Unnamed: 12': 'q2_r',
    'The goals and objectives of this course, as I understand them, are met at this time:': 'q3',
    'Unnamed: 14': 'q3_r',
    'The requirements and grading system for the course were clear:': 'q4',
    'Unnamed: 16': 'q4_r',
    'The instructors treated me with respect:': 'q5',
    'Unnamed: 18': 'q5_r',
    'The instructors were available to help when asked:': 'q6',
    'Unnamed: 20': 'q6_r',
    'The instructors were knowledgeable about the course material:': 'q7',
    'Unnamed: 22': 'q7_r',
    'The instructors were organized and prepared:': 'q8',
    'Unnamed: 24': 'q8_r',
    'I am satisfied with the format of the instruction:': 'q9',
    'Unnamed: 26': 'q9_r',
    'The pace of the course was conducive to my learning:': 'q10',
    'Unnamed: 28': 'q10_r',
    'The level of the course material was conducive to my learning:': 'q11',
    'Unnamed: 30': 'q11_r',
    'On average, how many hours per week have you spent, outside of the classroom, on this course?': 'q12',
    'Unnamed: 32': 'q12_r',
    'Are there any changes you would recommend for this course?': 'q13_r',
    'Did you have any concerns before beginning the bootcamp and if so, were those concerns alleviated?': 'q14',
    'Unnamed: 35': 'q14_r',
    'Who challenged you the most?': 'q15_r',
    'Who made you feel the most comfortable?': 'q16_r',
    'Who was your favorite guest instructor or speaker? Why?': 'q17_r',
    'Who was your LEAST favorite guest instructor or speaker? Why?': 'q18_r',
    'I had a good experience with myåÊBootcamp Success Manager (Sue):': 'qsue',
    'Unnamed: 41': 'qsue_r',
    'I had a good experience withåÊmy Pro Dev instructor (Daniel):': 'qdan',
    'Unnamed: 43': 'qdan_r',
    'I had a good experience with the Program Manager (Devonna):': 'qdev',
    'Unnamed: 45': 'qdev_r',
    'The STEMulus Center provided an effective learning environment:': 'qstem',
    'Unnamed: 47': 'q22_r',
    'Would you like to leave a review about your bootcamp experience?åÊIf so, please write a short testimonial. It may be used for our marketing purposes.': 'q23_r',
    'One a scale of 1-10, 1 being lowest and 10 highest, how likely areåÊyou toåÊ recommend this course to others?':'q1_1',
    'Did you have any concerns before beginning the bootcamp?': 'q14_s',
    'If you did have concerns before beginning the bootcamp, were those concerns alleviated?': 'q14_1',
    'Unnamed: 37' : 'q14_1_1',
    'Respondent ID': 'Respondent ID',
    'Collector ID': 'Collector ID',
    'Start Date': 'Start Date',
    'End Date': 'End Date',
    'Email Address': 'Email Address',
    'First Name': 'First Name',
    'Last Name': 'Last Name',
    'Custom Data 1': 'Custom Data 1',
    'I had a good experience with the Employer Relations Manager (Esteban):': 'qeste',
    'Unnamed: 49':'qstem_r',
    'Unnamed: 9' : 'q2_r',
    'Unnamed: 11' : 'q3_r',
    'Unnamed: 13' : 'q5_r',
    'Unnamed: 15' : 'q4_r',
    'Unnamed: 17' : 'q_s_1_r',
    'Unnamed: 19' : 'q6_s_r',
    'Unnamed: 21' : 'q8_s_r',
    'Unnamed: 23' : 'q9_r',
    'Unnamed: 25' : 'q10_r',
    'Unnamed: 27' : 'q11_r',
    'Unnamed: 29' : 'q_s_2_r',
    'Unnamed: 31' : 'q_s_3_r',
    'Unnamed: 34' : 'q14_r',
    'I had a good experience with my Student Success Specialist (Karen):' : 'qkar',
    'Unnamed: 38' : 'qkar_r',
    'I had a good experience with my Career Coach (Daniel):' : 'qdan',
    'Unnamed: 40' : 'qdan_r',
    'I had a good experience with the Senior Program Manager (Andrea):' : 'qand',
    'Unnamed: 42' : 'qand_r',
    'Unnamed: 44' : 'qstem_r',
    'Would you like to leave a testimonial? If we may use it for marketing purposes, please include your name.åÊ' : 'q23_r',
    'I understood the goals/objective of this course at the beginning of the course:' : 'q2',
    'The goals/objective of this course, as I understand them, are met at this time:' : 'q3',
    'My instructor treated me with respect:' : 'q5',
    'I felt free to ask questions in the course:' : 'q_s_1',
    'The instructor answered my questions when I asked them:' : 'q6_s',
    "The instructor's course presentations were organized:" : 'q8_s',
    'The textbook helped me to learn:' : 'q_s_2',
    'Homework assignments helped me better understand the course material:' : 'q_s_3',
    'The instructor was willing to work with me outside of the class:' : 'q_s_4',
    'Unnamed: 33' : 'q_s_4_r',
    'The instructor evaluated my learning based on course material presented and course requirements:' : 'q_s_5',
    'I have become more competent in this area due to this course:' : 'q_s_6',
    'Unnamed: 39' : 'q12_r',
    'List one or more aspects of the course that you liked:' : 'q_s_7_r'

    }

The above dictionary needs to be checked for human error. To test the above dictionary we need to make sure that all current column names are mapped to their new code. The below function will affirm that all column names in a given column have a mapping and return any that have been missed. 

In [ ]:
def check_column_names(dataframe):
  """Check if all column names in dataframe are in the dictionary as keys."""

  output = []
  for column in dataframe.columns:
    if column in column_names_mapped:
      continue
    else:
      output.append(column)
  if len(output) == 0:
    return 'All column names are found in dictionary'
  else:
    return output

In [ ]:
check_column_names(nlp1720)
check_column_names(nlp19)
check_column_names(nlp20)
check_column_names(nlp17)
check_column_names(nlp1417)

'All column names are found in dictionary'

Rename the columns in all datasets using the above mapping.

In [ ]:
nlp19.rename(column_names_mapped_nlp19, axis= 1, inplace=True)
nlp20.rename(column_names_mapped_nlp20, axis= 1, inplace=True)
nlp17.rename(column_names_mapped_nlp17, axis= 1, inplace=True)
nlp1417.rename(column_names_mapped_nlp1417, axis= 1, inplace=True)
nlp19.rename(column_names_mapped, axis= 1, inplace=True)
nlp20.rename(column_names_mapped, axis= 1, inplace=True)
nlp1720.rename(column_names_mapped, axis= 1, inplace=True)
nlp17.rename(column_names_mapped, axis= 1, inplace=True)
nlp1417.rename(column_names_mapped, axis= 1, inplace=True);

Check if all column names are mapped to codes.

In [ ]:
print(nlp1720.columns)
print(nlp19.columns)
print(nlp20.columns)
print(nlp17.columns)
print(nlp1417.columns)

Index(['Respondent ID', 'Collector ID', 'Start Date', 'End Date',
       'Email Address', 'First Name', 'Last Name', 'Custom Data 1', 'q1',
       'q1_1', 'q1_r', 'q2', 'q2_r', 'q3', 'q3_r', 'q4', 'q4_r', 'q5', 'q5_r',
       'q6', 'q6_r', 'q7', 'q7_r', 'q8', 'q8_r', 'q9', 'q9_r', 'q10', 'q10_r',
       'q11', 'q11_r', 'q12', 'q12_r', 'q13_r', 'q14', 'q14_r', 'q15_r',
       'q16_r', 'q17_r', 'q18_r', 'qsue', 'qsue_r', 'qdan', 'qdan_r', 'qdev',
       'qdev_r', 'qstem', 'q22_r', 'q23_r'],
      dtype='object')
Index(['Respondent ID', 'Collector ID', 'Start Date', 'End Date',
       'Email Address', 'First Name', 'Last Name', 'Custom Data 1', 'q1',
       'q1_1', 'q1_r', 'q2', 'q2_r', 'q3', 'q3_r', 'q4', 'q4_r', 'q5', 'q5_r',
       'q6', 'q6_r', 'q7', 'q7_r', 'q8', 'q8_r', 'q9', 'q9_r', 'q10', 'q10_r',
       'q11', 'q11_r', 'q12', 'q12_r', 'q13_r', 'q14_s', 'p14_s_r', 'q14_1',
       'q14_1_1', 'q15_r', 'q16_r', 'q17_r', 'q18_r', 'qsue', 'qsue_r', 'qdev',
       'qdev_r', 'qstem', 'qs

Append the three data sets into one unified data frame.

In [ ]:
combined_data = nlp1720.append([nlp19, nlp20, nlp17, nlp1417], ignore_index=True, sort=True)
combined_data.head()

,Collector ID,Custom Data 1,Email Address,End Date,First Name,Last Name,Respondent ID,Start Date,p14_s_r,q1,q10,q10_r,q11,q11_r,q12,q12_r,q13_r,q14,q14_1,q14_1_1,q14_1_r,q14_r,q14_s,q15_r,q16_r,q17_r,q18_r,q1_1,q1_r,q2,q22_r,q23_r,q2_r,q3,q3_r,q4,q4_r,q5,q5_r,q6,q6_r,q6_s,q6_s_r,q7,q7_r,q8,q8_r,q8_s,q8_s_r,q9,q9_r,q_s_1,q_s_1_r,q_s_2,q_s_2_r,q_s_3,q_s_3_r,q_s_4,q_s_4_r,q_s_5,q_s_5_r,q_s_6,q_s_6_r,q_s_7_r,qand,qand_r,qdan,qdan_r,qdev,qdev_r,qeste,qeste_r,qkar,qkar_r,qstem,qstem_r,qsue,qsue_r
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Response,Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Open-Ended Response,Response,NaN,NaN,NaN,PleaseåÊexplain why,NaN,Open-Ended Response,Open-Ended Response,Open-Ended Response,Open-Ended Response,Response,Why or why not?,Response,Other (please specify),Open-Ended Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),NaN,NaN,Response,Other (please specify),Response,Other (please specify),NaN,NaN,Response,Other (please specify),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Response,Other (please specify),Response,Other (please specify),NaN,NaN,NaN,NaN,Response,NaN,Response,Other (please specify)
1,175557115.0,NaN,NaN,9/12/19 10:52,NaN,NaN,7.025712e+09,9/12/19 10:23,NaN,Fullstack Web Development,Agree,The pace is quick. Don't blink or you'll miss ...,Agree,Addressing such a wide variety of levels of kn...,10-Jun,The potential rabbit hole to keep digging and ...,I think the capstones should come with a bit m...,No,NaN,NaN,NaN,NaN,NaN,"Marty with his ninja JavaScript skills, George...",Paul. He promised early on to work to earn my ...,Tough to pick between Andrea with the Project ...,There is no one at the bottom of this list.,Yes,I already do! I think this is a great opportun...,Agree Strongly,It's loud with ventilation and a dozen plus pe...,Deep Dive Coding Camps hold the potential to c...,They were clearly stated and with such an exte...,Agree Strongly,The road map was clearly stated and fairly wel...,Agree Strongly,"I appreciate the grading structure, as it depe...",Agree Strongly,The instructors (aka oracles) were all incredi...,Agree,"Even within a small cohort, sometimes it felt ...",NaN,NaN,Agree Strongly,It is hard to imagine how much more our instru...,Agree Strongly,It's interesting to see the mountain of curric...,NaN,NaN,Agree,Knowing that this was a transition class betwe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agree Strongly,Dan was insightful and candid. I believe his c...,Agree Strongly,Devonna is a calm port in the center of the st...,NaN,NaN,NaN,NaN,Agree,NaN,Agree Strongly,I am so happy Sue joined during our cohort! Sh...
2,175557115.0,NaN,NaN,9/12/19 10:51,NaN,NaN,7.025711e+09,9/12/19 10:24,NaN,Fullstack Web Development,Neutral,"It was very fast paced. At times, like drinkin...",Neutral,I think I could have been better prepared in b...,10-Jun,NaN,More foundations in Javascript.,Yes,NaN,NaN,NaN,NaN,NaN,Paul,Paul made me feel most comfortable. I enjoyed ...,I really enjoyed learning from Rochelle. She e...,NaN,Yes,However- I would recommend that people take so...,Agree,NaN,NaN,NaN,Agree,NaN,Agree Strongly,NaN,Agree Strongly,I was very pleased with the way I was treated.,Agree,Most of the time they could help.,NaN,NaN,Agree,They were very knowledgeable.,Neutral,There were some lectures that seemed to be hap...,NaN,NaN,Agree,I learned the best by doing.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agree,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,Agree Strongly,NaN,Agree Strongly,NaN
3,175557115.0,NaN,NaN,9/12/19 10:50,NaN,NaN,7.025711e+09,9/12/19 10:23,NaN,Fullstack Web Development,Neutral,"I came in knowing it would be very fast paced,...",Agree,NaN,16åÊ- 20,NaN,"As mentioned above in question 9, providing ju...",No,NaN,NaN,NaN,NaN,NaN,I think Paul challenged us the most by really ...,Brent a

In [ ]:
combined_data.shape

(304, 78)

###Delete Useles Columns and Rows

Drop unusable columns, and top row.

In [ ]:
combined_data.drop(['Collector ID', 'Custom Data 1', 'Email Address', 'End Date', 'First Name', 'Last Name', 'Respondent ID', 'Start Date', 'qkar_r', 'qand_r'], axis=1, inplace=True)

Drop unusable rows.

In [ ]:
hedder_rows = combined_data[combined_data['q3']=='Response'].index

In [ ]:
combined_data.drop(hedder_rows, axis=0, inplace=True)

###Clean Individual Columns

q12

In [ ]:
bad_str=combined_data['q12'].unique()

# check each unique value and replace with
# convert to label encoding

for idx,item in enumerate(combined_data['q12']):
  if item not in bad_str:
    continue
  elif item == '10-Jun' or item=='0 - 5' or item == '4.1 - 6' or item=='8+' or item=='6.1 - 8' or item=='0 - 2' or item=='2.1 - 4':
    combined_data['q12'][idx]='less than 10h'
  elif item =='15-Nov':
    combined_data['q12'][idx]='10h - 15h'
  elif item == '16åÊ- 20':
    combined_data['q12'][idx]='15h - 20h'
  elif item == '20+':
    combined_data['q12'][idx]= 'greater than 20h'

In [ ]:
combined_data['q12'].unique()

array(['less than 10h', '15h - 20h', '10h - 15h', 'greater than 20h'],
      dtype=object)

q1_1 conversion

Given the three datasets: nlp1720, nlp19, nlp20, there was a change to how responses were requested from binary (Y,N) to a Likert scale (1 - 10) for the question "Would you recommend this course to others?"

To address these differences it was decided to collapse the Likert scale into a Binary pattern.

| Likert Scale | Binary Value |
|-|-|
| <=5 |	N |
| >5 |	Y |


In [ ]:
combined_data['q1_1'].fillna('No Response', inplace=True)

In [ ]:
combined_data['q1_1'].unique()

array(['Yes', 'No Response', 'No', '9', '10', '8', '7', '2', '4', '5',
       '3', '6'], dtype=object)

In [ ]:
for idx,item in enumerate(combined_data['q1_1']):
  if item=='Yes' or item=='No' or item=='No Response':
    continue
  elif int(item)>5:
    combined_data['q1_1'][idx]='Yes'
  else:
    combined_data['q1_1'][idx] ='No'

In [ ]:
combined_data['q1_1'].unique()

array(['Yes', 'No Response', 'No'], dtype=object)

###Fill Nan values wtih 'No Response'

In [ ]:
clean_data = combined_data.fillna('No Response').copy()

Reset index

In [ ]:
clean_data.reset_index(drop=True, inplace=True)

In [ ]:
clean_data.head()

,p14_s_r,q1,q10,q10_r,q11,q11_r,q12,q12_r,q13_r,q14,q14_1,q14_1_1,q14_1_r,q14_r,q14_s,q15_r,q16_r,q17_r,q18_r,q1_1,q1_r,q2,q22_r,q23_r,q2_r,q3,q3_r,q4,q4_r,q5,q5_r,q6,q6_r,q6_s,q6_s_r,q7,q7_r,q8,q8_r,q8_s,q8_s_r,q9,q9_r,q_s_1,q_s_1_r,q_s_2,q_s_2_r,q_s_3,q_s_3_r,q_s_4,q_s_4_r,q_s_5,q_s_5_r,q_s_6,q_s_6_r,q_s_7_r,qand,qdan,qdan_r,qdev,qdev_r,qeste,qeste_r,qkar,qstem,qstem_r,qsue,qsue_r
0,No Response,Fullstack Web Development,Agree,The pace is quick. Don't blink or you'll miss ...,Agree,Addressing such a wide variety of levels of kn...,less than 10h,The potential rabbit hole to keep digging and ...,I think the capstones should come with a bit m...,No,No Response,No Response,No Response,No Response,No Response,"Marty with his ninja JavaScript skills, George...",Paul. He promised early on to work to earn my ...,Tough to pick between Andrea with the Project ...,There is no one at the bottom of this list.,Yes,I already do! I think this is a great opportun...,Agree Strongly,It's loud with ventilation and a dozen plus pe...,Deep Dive Coding Camps hold the potential to c...,They were clearly stated and with such an exte...,Agree Strongly,The road map was clearly stated and fairly wel...,Agree Strongly,"I appreciate the grading structure, as it depe...",Agree Strongly,The instructors (aka oracles) were all incredi...,Agree,"Even within a small cohort, sometimes it felt ...",No Response,No Response,Agree Strongly,It is hard to imagine how much more our instru...,Agree Strongly,It's interesting to see the mountain of curric...,No Response,No Response,Agree,Knowing that this was a transition class betwe...,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,Agree Strongly,Dan was insightful and candid. I believe his c...,Agree Strongly,Devonna is a calm port in the center of the st...,No Response,No Response,No Response,Agree,No Response,Agree Strongly,I am so happy Sue joined during our cohort! Sh...
1,No Response,Fullstack Web Development,Neutral,"It was very fast paced. At times, like drinkin...",Neutral,I think I could have been better prepared in b...,less than 10h,No Response,More foundations in Javascript.,Yes,No Response,No Response,No Response,No Response,No Response,Paul,Paul made me feel most comfortable. I enjoyed ...,I really enjoyed learning from Rochelle. She e...,No Response,Yes,However- I would recommend that people take so...,Agree,No Response,No Response,No Response,Agree,No Response,Agree Strongly,No Response,Agree Strongly,I was very pleased with the way I was treated.,Agree,Most of the time they could help.,No Response,No Response,Agree,They were very knowledgeable.,Neutral,There were some lectures that seemed to be hap...,No Response,No Response,Agree,I learned the best by doing.,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,No Response,Agree,No Response,Neutral,No Response,No Response,No Response,No Response,Agree Strongly,No Response,Agree Strongly,No Response
2,No Response,Fullstack Web Development,Neutral,"I came in knowing it would be very fast paced,...",Agree,No Response,15h - 20h,No Response,"As mentioned above in question 9, providing ju...",No,No Response,No Response,No Response,No Response,No Response,I think Paul challenged us the most by really ...,Brent and George were probably the people I fe...,Alonzo,"Captain Nick, I think he's a wonderful speaker...",Yes,No Response,Agree Strongly,"Love all the support that is available, having...",No Response,No Response,Agree,No Response,Agree,No Response,Agree,No Response,Agree,There were occasional times during capstone wo...,No Response,No Response,Agree,No Response,Agree,No Response,No Response,No Response,Disagree,I understand the program follows the tech fiel...,No Response,No Response,No Response,No Response,No Response,No Response,No Re

###Spell Check

In [ ]:
clean_data['q2_r']

0      They were clearly stated and with such an exte...
1                                            No Response
2                                            No Response
3                                            No Response
4                                            No Response
                             ...                        
294                                          No Response
295                                          No Response
296                                          No Response
297                                          No Response
298                                          No Response
Name: q2_r, Length: 299, dtype: object

In [ ]:
for entry in clean_data['q2_r']:
  if entry == 'No Response':
    continue
  else:
    sentence = tb(entry)
    if sentence.correct() == sentence:
      continue
    else:
      print(sentence)
      print(sentence.correct())

Sorta.. I probably should've done more research before signing up to this specific course. I'm definitely glad I finished, though!
Aorta.. I probably should've done more research before signing up to this specific course. I'm definitely glad I finished, though!
Adobe was mentioned, and that we would be doing a bit of game design.
Done was mentioned, and that we would be doing a bit of game design.
I thought I did understand in a general sort of way, but I wish that the reality of what a fullstack developer is, what a "win" or "success" at the bootcamp looks like, since I spent a large amount of time convinced that I was a failure, along with all the other bootcamp attendees
I thought I did understand in a general sort of way, but I wish that the reality of what a fullstack developer is, what a "win" or "success" at the bootcamp looks like, since I spent a large amount of time convinced that I was a failure, along with all the other bootcamp attended
The director and instructors made it

In [ ]:
for entry in clean_data['q2_r']:
  cell = entry.split()
  for word in cell:
    curent_word = Word(word)
    confidence = int(curent_word.spellcheck()[0][1])
    if confidence >= .8:
      continue
    else:
      print(curent_word.spellcheck())

[('They', 0.997467071935157), ('Hey', 0.0025329280648429585)]
[('time', 0.8657984144960362), ('times', 0.13363533408833522), ('timed', 0.0005662514156285391)]
[('To', 0.7805182471849138), ('So', 0.0818613485280152), ('No', 0.06370913037579705), ('Do', 0.04078144078144078), ('Go', 0.02455569122235789), ('O', 0.006973273639940307), ('Co', 0.000814000814000814), ('Oo', 0.0002442002442002442), ('Ho', 0.0002442002442002442), ('Fo', 0.0001085334418667752), ('Po', 8.14000814000814e-05), ('Wo', 2.71333604666938e-05), ('Vo', 2.71333604666938e-05), ('Mo', 2.71333604666938e-05), ('Lo', 2.71333604666938e-05)]
[('To', 0.7805182471849138), ('So', 0.0818613485280152), ('No', 0.06370913037579705), ('Do', 0.04078144078144078), ('Go', 0.02455569122235789), ('O', 0.006973273639940307), ('Co', 0.000814000814000814), ('Oo', 0.0002442002442002442), ('Ho', 0.0002442002442002442), ('Fo', 0.0001085334418667752), ('Po', 8.14000814000814e-05), ('Wo', 2.71333604666938e-05), ('Vo', 2.71333604666938e-05), ('Mo', 2.

In [ ]:
x = Word('No Respoonce')
x.spellcheck()

[('No Respoonce', 0.0)]

##Exploratory Data Analysis

In [ ]:
nlp20[0:2]

,Respondent ID,Collector ID,Start Date,End Date,Email Address,First Name,Last Name,Custom Data 1,q1,q1_1,q1_r,q2,q2_r,q3,q3_r,q4,q4_r,q5,q5_r,q6,q6_r,q7,q7_r,q8,q8_r,q9,q9_r,q10,q10_r,q11,q11_r,q12,q12_r,q13_r,q14_s,p14_s_r,q14_1,q14_1_r,q15_r,q16_r,q17_r,q18_r,qsue,qsue_r,qdev,qdev_r,qeste,qeste_r,qstem,qstem_r,q23_r
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Response,Response,Comments?,Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Open-Ended Response,Response,PleaseåÊexplain why,Response,PleaseåÊexplain why,Open-Ended Response,Open-Ended Response,Open-Ended Response,Open-Ended Response,Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Response,Other (please specify),Open-Ended Response
1,1.173612e+10,256829379.0,6/26/20 20:18,6/26/20 20:59,NaN,NaN,NaN,NaN,Fullstack Web Development,9,NaN,Agree,NaN,Agree,NaN,Agree,NaN,Agree Strongly,NaN,Agree,NaN,Agree,NaN,Neutral,NaN,Agree,NaN,Neutral,NaN,Agree,NaN,20+,NaN,More organized curriculum (seemed to jump arou...,Yes,Was concerned I wasn't going to get a full exp...,Yes,"Hard to know how it would have been on campus,...",George,Sue,Paul - I really enjoyed his take/attitude on t...,"If I had to pick, Carlos - really great to kno...",Agree Strongly,NaN,Agree,NaN,Agree,NaN,NaN,Fully remote bootcamp - no interaction with th...,NaN


In [ ]:
nlp1720.shape


(154, 49)

In [ ]:
nlp19.shape

(35, 49)

In [ ]:
nlp20.shape

(59, 51)

###Bootcamp Distribution --- Done

In [ ]:
column_names_mapped['Which bootcamp did you attend?']

'q1'

In [ ]:
#Bootcamp set
a=nlp1720['q1'].value_counts()
b=nlp19['q1'].value_counts()
c=nlp20['q1'].value_counts()

In [ ]:
#Concatenate all three lists together
bootcamps=pd.concat([a, b, c], axis=1)

In [ ]:
# Replace NAN and get total responses for each bootcamp
bootcamps=bootcamps.fillna(0)
bootcamps.loc[:,'Total']=bootcamps.sum(axis=1)

In [ ]:
# Cleanup result
bootcamps_clean=bootcamps.drop(['q1'], axis=1, inplace=True)
bootcamps=bootcamps.drop(labels=['Response'], axis=0)

In [ ]:
bootcamps

,Total
Fullstack Web Development,128.0
Java+Android,74.0
Digital Media,30.0
Microsoft .NET,3.0
Internet of Things (IoT),7.0
Data Science,3.0


### Addressing "Recommend" Responses

Given the three datasets: nlp1720, nlp19, nlp20, there was a change to how responses were requested from binary (Y,N) to a Likert scale (1 - 10) for the question "Would you recommend this course to others?" 

To address these differences it was decided to collapse the Likert scale into a Binary pattern. 

|Likert Scale <br>value| Binary <br>value|
|-|-|
| <=5 | N |
| >5 | Y|

To apply, a function will be constructed to perform the translation.

##Function to convert Likert to Binary


Determining distribution of responses on question
nlp1720: q1_1


In [ ]:
column_names_mapped['Would you recommend this course to others?']

'q1_1'

In [ ]:
nlp1720['q1_1'].value_counts()

Yes         143
No            8
Response      1
Name: q1_1, dtype: int64

In [ ]:
column_names_mapped['One a scale of 1-10, 1 being lowest and 10 highest, how likely areåÊyou toåÊ recommend this course to others?']

'q1_1'

In [ ]:
nlp19['q1_1'].value_counts()

10          14
9            9
8            5
7            3
4            1
2            1
Response     1
Name: q1_1, dtype: int64

In [ ]:
nlp20['q1_1'].value_counts()

10          32
9           10
8            8
7            3
3            3
6            1
5            1
Response     1
Name: q1_1, dtype: int64

In [ ]:
#review specific NO responses
nlp1720[nlp1720['q1_1']=='No']

,Respondent ID,Collector ID,Start Date,End Date,Email Address,First Name,Last Name,Custom Data 1,q1,q1_1,q1_r,q2,q2_r,q3,q3_r,q4,q4_r,q5,q5_r,q6,q6_r,q7,q7_r,q8,q8_r,q9,q9_r,q10,q10_r,q11,q11_r,q12,q12_r,q13_r,q14,q14_r,q15_r,q16_r,q17_r,q18_r,qsue,qsue_r,qdan,qdan_r,qdev,qdev_r,qstem,q22_r,q23_r
44,6.997736e+09,175405225.0,4/21/19 9:29,4/21/19 9:35,NaN,NaN,NaN,NaN,Java+Android,No,NaN,Neutral,NaN,Neutral,NaN,Neutral,NaN,Neutral,NaN,Disagree,Instructors gave greater time to certain stude...,Neutral,NaN,Neutral,NaN,Neutral,NaN,Neutral,NaN,Neutral,NaN,10-Jun,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Neutral,NaN,Neutral,NaN,Neutral,NaN,Neutral,NaN,NaN
47,6.997480e+09,175405225.0,4/19/19 11:30,4/19/19 12:05,NaN,NaN,NaN,NaN,Digital Media,No,I felt like digital media wasnÛªt held up to ...,Disagree,I was confused pretty early on. I felt like th...,Disagree,"Not really, there was a lot of ÛÏself learnin...",Disagree Strongly,We were not told how things would be graded. W...,Agree,NaN,Neutral,If I needed help with photoshop or illustrator...,Disagree,I feel like I didnÛªt learn anything I didnÛ...,Agree,We had a set path on what we were going to lea...,Disagree Strongly,I would have liked an instructor who showed pe...,Agree,NaN,Agree,NaN,15-Nov,NaN,A new instructor. HeÛªs nice and has a lot ...,No,"No, it looked very promising.",My fellow cohort mates.,Everyone made me feel welcome. ItÛªs a nice e...,"Nick, because of how passionate he seemed abou...",The gap inc. They seemed to know exactly wha...,Agree Strongly,"SheÛªs great, I felt like she really cared an...",Agree Strongly,Dan is dope.,Agree Strongly,NaN,Agree Strongly,NaN,NaN
69,6.967738e+09,171805496.0,12/14/18 9:57,12/14/18 11:04,NaN,NaN,NaN,NaN,Fullstack Web Development,No,"If they get a scholarship sure it free, but I...",Agree Strongly,NaN,Disagree,NaN,Agree,NaN,Neutral,(comment removed),Disagree Strongly,It got so bad for fun I started to keep track ...,Agree Strongly,NaN,Disagree Strongly,NaN,Disagree,NaN,Agree,NaN,Neutral,NaN,20+,NaN,I think people could get a lot more out of the...,No,NaN,NaN,NaN,NaN,NaN,Agree Strongly,NaN,Agree,NaN,NaN,NaN,Agree,NaN,I know I was harsh with some of my commits. I ...
74,6.967735e+09,171805496.0,12/14/18 10:30,12/14/18 10:53,NaN,NaN,NaN,NaN,Fullstack Web Development,No,There needs to be more organization in the cou...,Agree,NaN,Disagree,NaN,Agree,NaN,Agree,NaN,Agree,NaN,Agree,NaN,Disagree,NaN,Disagree,NaN,Neutral,NaN,Neutral,NaN,20+,NaN,"More organization,",No,NaN,Brent,Brent and Chloe,"Shihlin, the person who talked about the impos...",Big O. He was too advanced and went way over o...,Agree,NaN,Neutral,NaN,NaN,NaN,Neutral,Sometimes it was too noisy. I know you're tryi...,NaN
86,6.967693e+09,171805496.0,12/14/18 8:36,12/14/18 8:49,NaN,NaN,NaN,NaN,Fullstack Web Development,No,"Honestly, I do not think George is an effectiv...",Agree,NaN,Agree,NaN,Disagree,The rubrics are kind of vague. They say things...,Neutral,"Chloe, Brent, and Erik were super helpful. Geo...",Agree,"Mostly, but sometimes either one instructor or...",Agree,For the most part. I did get stuck on somethin...,Neutral,Assistant instructors were often late or left ...,Disagree Strongly,"No. The first 5 weeks were insanely easy, then...",Neutral,"No. WAY too slow for the first 6 weeks, then w...",Agree,NaN,10-Jun,NaN,"Replace George with an experienced instructor,...",Yes,Concerned because Dylan left and George replac...,George Kephart,Erik Clevenger,"Erik Clevenger. Super honest, always offering ...",The woman from CNM's IT sector. She started of...,Neutral,NaN,Agree,"He clearly works hard, but I think he's always...",NaN,NaN,Agree,NaN,I was stuck in minimum wage jobs and was start...
122,6.825121e+09,173298664.0,4/13/18 10:57,4/13/18 11:17,NaN,NaN,NaN,NaN,Java+Android,No,"Kotlin is the new Android, but Java is still i...",Agree Strongly,NaN,Agree Strongly,NaN,Agree Strongly,NaN,Agree Strongly,NaN,Agree Strongly,NaN,Agree Strongly,NaN,Agree Strongly,NaN,Agree Strongly,NaN,Agree S

In [ ]:
# review low - below 5 responses
nlp19[nlp19['q1_1']=='4']

,Respondent ID,Collector ID,Start Date,End Date,Email Address,First Name,Last Name,Custom Data 1,q1,q1_1,q1_r,q2,q2_r,q3,q3_r,q4,q4_r,q5,q5_r,q6,q6_r,q7,q7_r,q8,q8_r,q9,q9_r,q10,q10_r,q11,q11_r,q12,q12_r,q13_r,q14_s,p14_s_r,q14_1,q14_1_1,q15_r,q16_r,q17_r,q18_r,qsue,qsue_r,qdev,qdev_r,qstem,qstem_r,q23_r
23,1.120203e+10,247716749.0,12/6/19 10:51,12/6/19 10:54,NaN,NaN,NaN,NaN,Digital Media,4,NaN,Agree Strongly,NaN,Agree Strongly,NaN,Agree Strongly,NaN,Neutral,NaN,Disagree,NaN,Neutral,NaN,Neutral,NaN,Disagree,NaN,Neutral,NaN,Neutral,NaN,15-Nov,NaN,NaN,No,NaN,No,NaN,One of the instructors was somewhat disrespectful,The classmates,The motion capture - it was something I had al...,There were only a handful,Agree,NaN,Agree,NaN,Neutral,NaN,NaN


In [ ]:
nlp20[nlp20['q1_1']=='5']

,Respondent ID,Collector ID,Start Date,End Date,Email Address,First Name,Last Name,Custom Data 1,q1,q1_1,q1_r,q2,q2_r,q3,q3_r,q4,q4_r,q5,q5_r,q6,q6_r,q7,q7_r,q8,q8_r,q9,q9_r,q10,q10_r,q11,q11_r,q12,q12_r,q13_r,q14_s,p14_s_r,q14_1,q14_1_r,q15_r,q16_r,q17_r,q18_r,qsue,qsue_r,qdev,qdev_r,qeste,qeste_r,qstem,qstem_r,q23_r
10,1.172508e+10,256829379.0,6/23/20 21:27,6/23/20 23:06,NaN,NaN,NaN,NaN,Fullstack Web Development,5,This is a good course that introduces a hand f...,Neutral,I had higher expectations that I would be able...,Disagree Strongly,I don't consider myself a web developer but I ...,Disagree,I never felt informed in advance about the req...,Agree,I felt I could speak up when needed during lec...,Agree,The instructors did a reasonably good job at t...,Agree,NaN,Neutral,I was a little disappointed with the instructo...,Neutral,Mostly. In retrospect certain technologies co...,Neutral,"The pace of the course was fast, which was exp...",Neutral,NaN,16åÊ- 20,Late nights and early mornings were common and...,I would recommend posting the assignments in a...,Yes,I was concerned about falling behind and/or no...,Yes,I felt like I was able to keep pace with the c...,I challenged myself the most by dedicating mys...,Kathleen. She was very helpful and took the t...,All the guest speakers/instructors were okay. ...,Least favorite speaker was Dylan. I felt like...,Agree Strongly,The weekly 5 minute meetings should become reg...,Agree,I only meet her once during the interview and ...,Agree,I enjoyed his personal cohort touch and the wa...,Neutral,NA. I was only at the STEMulus center once fo...,Like all education it's way over priced by at ...


##Data Proccessing

##Data Visualization